In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from statistics import mode
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
%config InlineBackend.figure_format = 'retina'
from sklearn.cluster import KMeans
import random
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('sentiment_data/grand_df.csv', index_col=0)

In [3]:
count = df.groupby('primary_artist')[['title']].count().reset_index()

In [4]:
count.head()

,primary_artist,title
0,2 Chainz,250
1,3OH!3,67
2,5 Seconds of Summer,102
3,50 Cent,348
4,A Great Big World,29


In [5]:
remove = count[count['title'] < 50].primary_artist.tolist()

In [6]:
df.head()

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url
0,Boy In Detention (Mixtape),14.0,13.0,/songs/62387,5.0,7.0,12.0,"Beats I be servin', Saks Fifth splurgin' Diam...",11.0,-0.9809,24.0,Chris Brown,2011-08-05,5.0,http://api.genius.com/songs/62387,11.0,100%,17.0,https://genius.com/Chris-brown-100-lyrics
1,Boy In Detention (Mixtape),19.0,1.0,/songs/54007,16.0,15.0,1.0,We in the motherfuckin' buildin' 100 fuckin...,20.0,-0.9967,13.0,Chris Brown,2011-08-05,16.0,http://api.genius.com/songs/54007,1.0,100 Bottles,4.0,https://genius.com/Chris-brown-100-bottles-lyrics
2,X,2.0,4.0,/songs/516349,2.0,1.0,6.0,You got me doing 101 on the 101 Don't go nowh...,6.0,0.9516,11.0,Chris Brown,2014-09-16,2.0,http://api.genius.com/songs/516349,2.0,101 (Interlude),6.0,https://genius.com/Chris-brown-101-interlude-l...
3,None,0.0,0.0,/songs/3103904,0.0,4.0,0.0,Ain't no way we could ignore this fire When I...,3.0,0.8592,4.0,Chris Brown,2017-05-31,2.0,http://api.genius.com/songs/3103904,0.0,10 Feet,2.0,https://genius.com/Chris-brown-10-feet-lyrics
4,None,20.0,17.0,/songs/57217,11.0,12.0,14.0,"Welcome to the Cypher Myself, K-Mac, Ace Hood...",23.0,0.9745,39.0,Chris Brown,2019-04-04,9.0,http://api.genius.com/songs/57217,17.0,2011 BET Cypher Chris Brown and Friends,25.0,https://genius.com/Chris-brown-2011-bet-cypher...


In [50]:
df[df.primary_artist == 'Don Omar']

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url
24892,None,0.0,0.0,/songs/2091417,0.0,0.0,0.0,Ahora son mejor que yo Ujum ujum Y estan ranki...,1.0,-0.8985,1.0,Don Omar,2010-09-07,1.0,http://api.genius.com/songs/2091417,0.0,Ahora son mejor,1.0,https://genius.com/Don-omar-ahora-son-mejor-ly...
24893,Don Omar Presents MTO²: New Generation,5.0,0.0,/songs/1801902,4.0,4.0,0.0,A lo mejor ya es tarde Para que tu estés conmi...,6.0,-0.8658,1.0,Don Omar,2012-10-02,3.0,http://api.genius.com/songs/1801902,0.0,A Lo Mejor Ya Es Tarde,1.0,https://genius.com/Don-omar-a-lo-mejor-ya-es-t...
24894,King of Kings,0.0,0.0,/songs/1085299,0.0,0.0,0.0,Amarga vida Tu que me has maltratao tanto Yo q...,1.0,-0.9859,0.0,Don Omar,2006-05-23,1.0,http://api.genius.com/songs/1085299,0.0,Amarga Vida,0.0,https://genius.com/Don-omar-amarga-vida-lyrics
24895,None,3.0,1.0,/songs/1432850,3.0,2.0,0.0,"""Bandoleros Reloaded""! Segunda edición! Don, e...",3.0,-0.9706,3.0,Don Omar,2010-09-07,2.0,http://api.genius.com/songs/1432850,0.0,Anda Sola,3.0,https://genius.com/Don-omar-anda-sola-lyrics
24896,King of Kings,0.0,0.0,/songs/63815,0.0,0.0,0.0,"Hey (Hey), hey (Hey) ¿Quiénes son? Eliel Do...",2.0,-0.9183,1.0,Don Omar,2006-05-02,2.0,http://api.genius.com/songs/63815,0.0,Angelito,1.0,https://genius.com/Don-omar-angelito-lyrics
24897,None,3.0,1.0,/songs/2120527,3.0,2.0,1.0,Vuelve ie ie... Vuelve ie ie... Que el dolor m...,9.0,-0.5951,2.0,Don Omar,2010-09-07,9.0,http://api.genius.com/songs/2120527,0.0,Anque te fuiste,2.0,https://genius.com/Don-omar-anque-te-fuiste-ly...
24898,MTO²: New Generation,1.0,0.0,/songs/1627344,1.0,1.0,0.0,Si me le pego ella rápido me besa Y si en lo o...,1.0,-0.8330,1.0,Don Omar,2012-06-04,1.0,http://api.genius.com/songs/1627344,0.0,Así es que es,0.0,https://genius.com/Don-omar-asi-es-que-es-lyrics
24899,The Last Don II,0.0,0.0,/songs/3239318,0.0,0.0,0.0,'Cause I remember cuando me bailabas sexy baj...,2.0,-0.9856,0.0,Don Omar,2015-06-16,1.0,http://api.genius.com/songs/3239318,0.0,Bailando Bajo El Sol,0.0,https://genius.com/Don-omar-bailando-bajo-el-s...
24900,King of Kings,0.0,0.0,/songs/1197383,0.0,0.0,0.0,Son esos ojos de leona cazando por la disco Sí...,3.0,0.8706,4.0,Don Omar,2006-05-23,1.0,http://api.genius.com/songs/1197383,0.0,Bailando Sola,0.0,https://genius.com/Don-omar-bailando-sola-lyrics
24901,King of Kings,0.0,0.0,/songs/1368543,0.0,0.0,7.0,Ohhhhh no Ohhhhh no Ohhhhh na na na na na na...,0.0,0.5242,18.0,Don Omar,2006-05-23,0.0,http://api.genius.com/songs/1368543,0.0,Belly Danza,9.0,https://genius.com/Don-omar-belly-danza-lyrics


In [7]:
for artist in remove:
    df.drop(df[df['primary_artist'] == artist].index, axis=0, inplace=True)

In [23]:
df_normalized = df.groupby('primary_artist')[['anger', 'positive', 'negative', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']].sum()
df_normalized.head()

,anger,positive,negative,anticipation,disgust,fear,joy,sadness,surprise,trust
primary_artist,,,,,,,,,,
2 Chainz,3780.0,5619.0,5550.0,2781.0,2642.0,2895.0,2637.0,2572.0,1530.0,3623.0
3OH!3,590.0,1134.0,1002.0,471.0,430.0,532.0,559.0,489.0,390.0,620.0
5 Seconds of Summer,492.0,1396.0,1294.0,699.0,414.0,643.0,683.0,781.0,390.0,887.0
50 Cent,6825.0,8158.0,10116.0,3922.0,4666.0,5451.0,3768.0,4349.0,3084.0,5358.0
A$AP Rocky,2176.0,3241.0,3561.0,1617.0,1662.0,1894.0,1601.0,1617.0,1084.0,2110.0


In [24]:
sums = df_normalized['anger'] + df_normalized['positive'] + df_normalized['negative'] + df_normalized['anticipation'] + df_normalized['disgust'] + df_normalized['fear'] + df_normalized['joy'] + df_normalized['sadness'] + df_normalized['surprise'] + df_normalized['trust']
df_normalized['sum'] = sums

In [25]:
for col in df_normalized.columns:
        df_normalized[col] = df_normalized[col].apply(int)

In [26]:
cols = list(df_normalized.columns)
cols.remove('sum')
for col in cols:
    df_normalized[col] = df_normalized[col] / df_normalized['sum']

In [27]:
df_normalized = df_normalized.reset_index()
df_normalized.head()

,primary_artist,anger,positive,negative,anticipation,disgust,fear,joy,sadness,surprise,trust,sum
0,2 Chainz,0.112403,0.167088,0.165036,0.082696,0.078563,0.086086,0.078414,0.076482,0.045496,0.107734,33629
1,3OH!3,0.094901,0.182403,0.161171,0.075760,0.069165,0.085572,0.089915,0.078655,0.062731,0.099727,6217
2,5 Seconds of Summer,0.064071,0.181795,0.168512,0.091027,0.053913,0.083735,0.088944,0.101706,0.050788,0.115510,7679
3,50 Cent,0.122538,0.146471,0.181626,0.070417,0.083775,0.097869,0.067652,0.078083,0.055371,0.096199,55697
4,A$AP Rocky,0.105821,0.157613,0.173175,0.078636,0.080825,0.092107,0.077858,0.078636,0.052716,0.102611,20563


In [52]:
sentiment = 'negative'

df_normalized[['primary_artist', sentiment]].sort_values(by=sentiment, ascending=False)

,primary_artist,negative
141,Lil Jon & The East Side Boyz,0.195120
70,Eminem,0.194497
63,Disturbed,0.193481
60,Desiigner,0.189748
86,Green Day,0.189155
144,Linkin Park,0.188787
168,Muse,0.185058
140,Lil Dicky,0.183363
3,50 Cent,0.181626
207,ScHoolboy Q,0.179805
